# Course Creation Example

This notebook demonstrates how to use the `CourseCreator` to generate course ideas and outlines.

## Import and instantiate the CourseCreator class

Initialize a `CourseCreator` instance. All course creation is performed through this object.

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()

# Check the key
key = os.getenv("OPENAI_API_KEY")
print(f"Key: {key[:8]}...{key[-4:]}")
# Test a simple API call
client = OpenAI()
try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # cheapest model
        messages=[{"role": "user", "content": "Say hello"}],
        max_tokens=5
    )
    print(f"Success! Response: {response.choices[0].message.content}")
except Exception as e:
    print(f"Error: {e}")

Key: sk-proj-...gpoA
Success! Response: Hello! How can I


In [4]:
from openai import OpenAI
client = OpenAI()

models_to_test = ["gpt-4o", "o3-mini", "gpt-4.1"]

for model in models_to_test:
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": "Hi"}],
            #max_tokens=5
        )
        print(f"✓ {model}: works")
    except Exception as e:
        print(f"✗ {model}: {e}")

✓ gpt-4o: works
✓ o3-mini: works
✓ gpt-4.1: works


In [13]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

key = os.getenv("GOOGLE_API_KEY")
if key:
    print(f"Key: {key[:8]}...{key[-4:]}")

    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=key)
    response = llm.invoke("Say hello")
    print(f"Success! Response: {response.content}")
else:
    print("No GOOGLE_API_KEY found in .env")
    print("Get one free at: https://aistudio.google.com/apikey")

Key: AIzaSyDT...zPOA
Success! Response: Hello!


In [13]:
from denario.course import CourseCreator

creator = CourseCreator(project_dir="./course_project", clear_project_dir=False)

Run directory: ./course_project/20251126_132428


## Set the course topic, audience, and duration

Specify the subject you want to teach, who the course is for, and how long it will be.

In [ ]:
creator.set_topic(
    topic="ChatGPT Professional Proficiency: From Basics to Advanced Features",
    audience="Business professionals seeking deep proficiency without coding. They want to go from basics in LLMs such as prompt and context engineering to understand precise tools and responsible AI practices.",
    duration="2 days (8 hours total)"
)

creator.show_topic()


Topic: Advanced ChatGPT Power User Training: Mastering Prompt and Context, Model Selection and Personalization
Target Audience: Business professionals seeking deep proficiency without coding. They want to move beyond basics to understand precise tools (Code Interpreter, Web Search, Deep Research), responsible AI practices (handling hallucinations, data privacy), and get an introduction to next-level concepts like RAG and AI Agents.
Duration: 2 days (8 hours total)


## Generate a course idea

Use the `idea_maker` and `idea_hater` agents to brainstorm and refine course ideas.

In [15]:
# Using gpt-4o-mini (available on lower OpenAI tiers)
creator.generate_idea()
#creator.generate_idea(
#    idea_maker_model="gpt-4o-mini",
#    idea_hater_model="gpt-4o-mini",
#    planner_model="gpt-4o-mini",
#    plan_reviewer_model="gpt-4o-mini",
#    orchestration_model="gpt-4o-mini",
#    formatter_model="gpt-4o-mini",
#)
# Original models (require higher OpenAI tier):
# creator.generate_idea(
#     idea_maker_model="gpt-4o",
#     idea_hater_model="o3-mini",
#     planner_model="gpt-4o",
#     plan_reviewer_model="o3-mini",
#     orchestration_model="gpt-4.1",
#     formatter_model="o3-mini",
# )
creator.show_idea()

Generating course idea...
Created context directory:  /Users/elenahernandez/projects/agents/Denario/examples/course_project/20251126_132428/idea_generation_output/context

Topic: Advanced ChatGPT Power User Training: Mastering Prompt and Context, Model Selection and Personalization
Target Audience: Business professionals seeking deep proficiency without coding. They want to move beyond basics to understand precise tools (Code Interpreter, Web Search, Deep Research), responsible AI practices (handling hallucinations, data privacy), and get an introduction to next-level concepts like RAG and AI Agents.
Duration: 2 days (8 hours total)


--------------------------------------------------------------------------------
Setting agents for the session...

             Model       agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
gpt-4.1-2025-04-14 plan_setter 0.00119            545                 13           558

------------------------------------------------------------------


Course Idea:
	* Mastering ChatGPT for Business: Advanced Tools and Techniques
		- This course is designed to empower business professionals with advanced ChatGPT tools, focusing on practical applications without the need for coding.
		- Participants will learn to leverage tools like Code Interpreter and Web Search, enhancing their decision-making processes with data-driven insights.
		- The course emphasizes responsible AI practices, including managing hallucinations and ensuring data privacy, crucial for ethical business operations.
		- Interactive simulations and role-play scenarios will be used to reinforce learning, ensuring participants can apply their knowledge in real-world business contexts.
		- By the end of the course, attendees will have a comprehensive understanding of how to integrate advanced AI tools into their business strategies effectively.

        

## Generate a course outline

Use the `researcher` agent to create a detailed course outline with modules, learning goals, and takeaways.

In [16]:
# Using gpt-4o-mini (available on lower OpenAI tiers)
creator.generate_outline()
#creator.generate_outline(
#    researcher_model="gpt-4o-mini",
#    planner_model="gpt-4o-mini",
#    plan_reviewer_model="gpt-4o-mini",
#    orchestration_model="gpt-4o-mini",
#    formatter_model="gpt-4o-mini", 
#)
# Original models (require higher OpenAI tier):
# creator.generate_outline(
#     researcher_model="gpt-4.1-2025-04-14",
#     planner_model="gpt-4.1-2025-04-14",
#     plan_reviewer_model="o3-mini",
#     orchestration_model="gpt-4.1",
#     formatter_model="o3-mini",
# )
creator.show_outline()

Generating course outline...
Created context directory:  /Users/elenahernandez/projects/agents/Denario/examples/course_project/20251126_132428/course_outline_generation_output/context
Generate the course outline based on the course idea provided above.

--------------------------------------------------------------------------------
Setting agents for the session...

             Model       agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
gpt-4.1-2025-04-14 plan_setter 0.00135            645                  8           653

--------------------------------------------------------------------------------
Plan constraints have been logged.

--------------------------------------------------------------------------------

Calling planner...

             Model   agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
gpt-4.1-2025-04-14 planner 0.00488            860                395          1255
**Plan:**

- Step 1:
    * sub-task: Analyze the course idea to identi

# Mastering ChatGPT for Business: Advanced Tools and Techniques

### Overview:
This hands-on course is designed for business professionals who want to integrate advanced ChatGPT tools into their daily work—no coding required. Across seven scaffolded modules, participants will learn to leverage features like the Code Interpreter and Web Search for data-driven decision-making, while mastering responsible AI practices. Interactive simulations and role-play scenarios ensure that learning is practical, ethical, and directly applicable to real-world business challenges.

---

## 1. Introduction to ChatGPT in Business Contexts

### Description:
This opening module establishes a foundational understanding of generative AI and ChatGPT, focusing on their relevance and transformative potential in business environments. Participants will explore key business applications, set personal learning objectives, and engage in an interactive discussion to surface current and anticipated AI use cases within their organizations.

### Takeaways:
- Clear understanding of ChatGPT’s capabilities and limitations in business.
- Awareness of diverse business scenarios where ChatGPT can add value.
- Personalized learning objectives aligned with professional roles.

### Learning Goals:
- Participants identify core functionalities of ChatGPT relevant to business.
- Participants describe potential business use cases for generative AI.
- Participants recognize the scope and limitations of ChatGPT in organizational contexts.

### Exercise Description:
Icebreaker and group discussion: 15 minutes, Debrief: 5 minutes

---

## 2. Navigating ChatGPT Advanced Tools

### Description:
This module provides hands-on experience with ChatGPT’s advanced features, including the Code Interpreter (Advanced Data Analysis) and Web Search. Participants will learn to navigate the interface, upload and process data, and utilize web search to enhance business insights, building confidence in tool proficiency.

### Takeaways:
- Practical skills in using ChatGPT’s advanced tools for business tasks.
- Ability to upload, process, and interpret business data.
- Enhanced capability to source and validate information using web search.

### Learning Goals:
- Participants navigate the ChatGPT interface and access advanced features.
- Participants demonstrate uploading and analyzing data with the Code Interpreter.
- Participants apply web search capabilities to gather and verify business information.

### Exercise Description:
Guided walkthrough and paired tool exploration: 25 minutes, Debrief: 10 minutes

---

## 3. Prompt Engineering for Business Outcomes

### Description:
Focusing on the art and science of prompt engineering, this module equips participants to craft, refine, and adapt prompts for a variety of business scenarios. Through a collaborative workshop, participants will practice designing prompts that yield accurate, actionable outputs, and learn strategies for iterative improvement.

### Takeaways:
- Mastery of prompt formulation tailored to business needs.
- Techniques for refining prompts to improve output quality.
- Peer-reviewed examples of effective prompt strategies.

### Learning Goals:
- Participants formulate effective prompts for specific business objectives.
- Participants refine prompts based on output evaluation and feedback.
- Participants distinguish between prompt strategies for different business scenarios.

### Exercise Description:
Prompt design workshop with peer feedback: 30 minutes, Debrief: 10 minutes

---

## 4. Data-Driven Decision Making with ChatGPT

### Description:
This module immerses participants in the practical application of ChatGPT’s Code Interpreter for business analytics. Participants will upload sample datasets, generate visualizations, and extract actionable insights, culminating in a simulation where they present findings to inform business decisions.

### Takeaways:
- Confidence in using ChatGPT for data analysis and visualization.
- Ability to interpret and communicate data-driven insights.
- Experience integrating AI-generated analytics into decision-making processes.

### Learning Goals:
- Participants analyze business data using the Code Interpreter.
- Participants generate and interpret visualizations to support business cases.
- Participants present data-driven recommendations based on AI analysis.

### Exercise Description:
Simulation: data analysis and presentation: 35 minutes, Debrief: 10 minutes

---

## 5. Responsible AI Use and Managing Hallucinations

### Description:
Addressing the ethical dimensions of AI, this module focuses on recognizing and mitigating hallucinations (inaccurate or fabricated outputs), ensuring data privacy, and implementing responsible AI practices. Through a role-play scenario, participants will navigate a business case involving AI-generated misinformation and privacy concerns.

### Takeaways:
- Skills to identify and manage AI hallucinations in business outputs.
- Understanding of data privacy best practices when using AI tools.
- Frameworks for responsible and ethical AI adoption in business.

### Learning Goals:
- Participants identify hallucinations and inaccuracies in AI-generated content.
- Participants evaluate the reliability and validity of AI outputs.
- Participants explain and apply data privacy best practices in business contexts.
- Participants navigate ethical dilemmas using responsible AI frameworks.

### Exercise Description:
Role-play scenario: managing AI misinformation and privacy: 30 minutes, Debrief: 10 minutes

---

## 6. Integrating ChatGPT into Business Workflows

### Description:
This module synthesizes technical and ethical competencies, guiding participants through the process of mapping ChatGPT tools to real business workflows. Emphasis is placed on change management, stakeholder communication, and designing sustainable AI integration strategies. Teams will collaborate to develop and present an AI integration plan for a hypothetical business challenge.

### Takeaways:
- Blueprint for integrating ChatGPT into existing business processes.
- Strategies for effective change management and stakeholder engagement.
- Experience designing and presenting AI-enhanced workflow solutions.

### Learning Goals:
- Participants map ChatGPT tools to specific business processes.
- Participants create an AI integration plan addressing technical and ethical considerations.
- Participants demonstrate effective communication of AI strategies to stakeholders.
- Participants evaluate potential challenges and solutions in AI adoption.

### Exercise Description:
Group simulation: AI integration planning and presentation: 40 minutes, Debrief: 15 minutes

---

## 7. Capstone Simulation and Action Planning

### Description:
In the culminating module, participants engage in a comprehensive simulation that requires the application of all skills and ethical frameworks developed throughout the course. Following the simulation, participants will formulate personalized action plans for integrating advanced ChatGPT tools into their own business contexts, with peer review to reinforce accountability and support.

### Takeaways:
- Real-world experience applying advanced ChatGPT tools and ethical practices.
- Personalized action plan for post-course AI integration.
- Peer feedback to refine strategies and ensure sustained impact.

### Learning Goals:
- Participants synthesize technical and ethical competencies in a real-world scenario.
- Participants formulate actionable plans for AI adoption in their organizations.
- Participants evaluate and refine their strategies through peer review.

### Exercise Description:
Capstone simulation and action planning: 45 minutes, Peer review and debrief: 15 minutes

---

**End of Course Outline**

## Alternative: Set idea manually

If you already have a course idea, you can set it directly and skip the idea generation step.

In [ ]:
# creator.set_idea(idea=r"""
# Course Title: Practical Machine Learning for Business Decisions
# 
# This hands-on course teaches business professionals how to leverage machine learning
# to make data-driven decisions. Participants will learn to build, evaluate, and deploy
# simple ML models using Python and scikit-learn. The course focuses on practical
# applications like customer segmentation, sales forecasting, and churn prediction.
# No advanced math required - emphasis is on intuition and implementation.
# """)
# 
# creator.generate_outline()
# creator.show_outline()

## Full workflow in one call

Alternatively, run the entire workflow (idea + outline generation) with a single method call.

In [ ]:
# creator2 = CourseCreator(project_dir="./another_course", clear_project_dir=True)
# creator2.set_topic(
#     topic="Data Visualization with Python",
#     audience="Data analysts familiar with Excel",
#     duration="1 day (4 hours)"
# )
# creator2.run()  # Generates both idea and outline
# creator2.show_outline()